In [1]:
# Achtung normalize und invert sind etwas anders weil pos schon auf 0,1
# Alle Trajektorien sind in einem file für die 6 input arten
# 900 Traj mit je 500 zeitschritten
# die Testtrajektorien sind immer die Enden der 30 Trajektorien (6*5) und 7mal länger

# Vorher T  = [0 , 0.0112] 500 steps
# Jetzt T = [0, 0.0224] 500 steps 
# => Schrittweite verdoppelt    
# Modell normal trainieren und dann bei Vergleich zu Simulink die Simulink Daten interpolieren damit sie gleich lang sind!

In [ ]:
!pip install psutil gputil

In [ ]:
import platform
import psutil
import GPUtil
import os

def get_system_info():
    print("===== SYSTEM INFORMATION =====")
    print(f"System: {platform.system()}")
    print(f"Node Name: {platform.node()}")
    print(f"Release: {platform.release()}")
    print(f"Version: {platform.version()}")
    print(f"Machine: {platform.machine()}")
    print(f"Processor: {platform.processor()}")
    print(f"Python Version: {platform.python_version()}")
    print(f"Architecture: {' '.join(platform.architecture())}")
    print(f"Hostname: {platform.node()}")
    print()

def get_memory_info():
    print("===== MEMORY INFORMATION =====")
    svmem = psutil.virtual_memory()
    print(f"Total Memory: {convert_size(svmem.total)}")
    print(f"Available Memory: {convert_size(svmem.available)}")
    print(f"Used Memory: {convert_size(svmem.used)}")
    print(f"Memory Percentage: {svmem.percent}%")
    print()

def get_cpu_info():
    print("===== CPU INFORMATION =====")
    print(f"Physical cores: {psutil.cpu_count(logical=False)}")
    print(f"Total cores: {psutil.cpu_count(logical=True)}")
    print(f"Max Frequency: {psutil.cpu_freq().max:.2f}Mhz")
    print(f"Min Frequency: {psutil.cpu_freq().min:.2f}Mhz")
    print(f"Current Frequency: {psutil.cpu_freq().current:.2f}Mhz")
    print(f"CPU Usage Per Core: {psutil.cpu_percent(percpu=True)}")
    print(f"Total CPU Usage: {psutil.cpu_percent()}%")
    print()

def get_disk_info():
    print("===== DISK INFORMATION =====")
    partitions = psutil.disk_partitions()
    for partition in partitions:
        print(f"Device: {partition.device}")
        print(f"  Mountpoint: {partition.mountpoint}")
        print(f"  File system type: {partition.fstype}")
        partition_usage = psutil.disk_usage(partition.mountpoint)
        print(f"  Total Size: {convert_size(partition_usage.total)}")
        print(f"  Used: {convert_size(partition_usage.used)}")
        print(f"  Free: {convert_size(partition_usage.free)}")
        print(f"  Percentage: {partition_usage.percent}%")
    print()

def get_gpu_info():
    print("===== GPU INFORMATION =====")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU Name: {gpu.name}")
        print(f"GPU Load: {gpu.load*100}%")
        print(f"GPU Free Memory: {convert_size(gpu.memoryFree)}")
        print(f"GPU Used Memory: {convert_size(gpu.memoryUsed)}")
        print(f"GPU Total Memory: {convert_size(gpu.memoryTotal)}")
        print(f"GPU Temperature: {gpu.temperature} °C")
        print(f"GPU Driver Version: {gpu.driver}")
        print()

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(os.log(size_bytes, 1024))
    p = os.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_name[i]}"

if __name__ == "__main__":
    get_system_info()
    get_cpu_info()
    get_memory_info()
    get_disk_info()
    get_gpu_info()


In [6]:
from meas_get_data import *
from meas_test_func_fs import * 
from meas_get_data import *
import os
torch.set_default_dtype(torch.float64)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
path = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
data =get_data(path, num_inits=100)
data.size()

path = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\Test-Messdaten_30traj_7times500steps.csv"
test_data =get_data(path, num_inits=100)
test_data.size()

In [ ]:
visualise(normalize_invert(test_data),3)
visualise(normalize_invert(test_data),4)
visualise(normalize_invert(test_data),5)

In [ ]:
visualise(normalize_invert(data),89)
visualise(normalize_invert(data),91)
visualise(normalize_invert(data),92)

In [ ]:

params_lstm =    {
                        "window_size" : 16,
                        "h_size" : 8,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,

                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 0,
                        "epochs" : 1000,
                        "test_every_epochs" : 100,

                        "T_max" : 500,

                        "experiment_number" : np.random.randint(0,1000)
                        }


if os.name == "nt":
    path_train_data=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
    path_test_data = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\TEST-Messdaten_30traj_7times500steps.csv"

else:
    path_train_data=r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/messdaten_900traj_500steps.csv"
    path_test_data = r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/TEST-Messdaten_30traj_7times500steps.csv"

test_data = get_data(path_test_data,num_inits=100)
train_data = get_data(path_train_data,num_inits=100)

 

model_lstm = OR_LSTM(input_size=4, hidden_size=params_lstm["h_size"], out_size=2, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)

path_lstm=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\or_lstm_0_3.pth"
model_lstm.load_state_dict(torch.load(path_lstm, map_location=torch.device(device)))

#test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 1, fix_random=True)
test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 5, fix_random=True)